In [12]:
import websocket
import json
import pandas as pd
import time
try:
    import thread
except ImportError:
    import _thread as thread
import time


coin = "btcusd" #Set you coin
init = 1
lst = None
header = pd.DataFrame({'timestamps':[],'price':[],'count':[],'amount':[]})


def on_message(ws, message):
    global init, coin
    result = json.loads(message)
    if (init <= 2):
        #file path, one for initial data, one for transaction records
        #the folder needs to be created before the program
        init += 1
    elif init <= 3: 
        timestamps = time.time() 
        #header.to_csv(coin+'.csv', header=True,index=False)
        header.to_csv(coin+str(timestamps)+'.csv', header=True,index=False)
        for events in result[1]:
            data = pd.DataFrame({'timestamps':timestamps,'orderID':[events[0]],'price':[events[1]],'amount':[events[2]]}) 
            data.to_csv(coin+str(timestamps)+'.csv',mode= 'a', header=False, index=False)
        ws.close()
        init += 1
    else:
        events = result[1]
        if (len(events)==3):   
            events = pd.DataFrame({'timestamps':[time.time()],'orderID':[events[0]],'price':[events[1]],'amount':[events[2]]}) 
            events.to_csv(coin+'.csv', mode= 'a', header=False, index=False)
def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    ws.send('{ "event": "subscribe", "channel": "book",  "symbol": "tBTCUSD", "len":"100"}')

if __name__ == "__main__":
    websocket.enableTrace(True)
    #url here, add any other options according to API document
    ws = websocket.WebSocketApp("wss://api-pub.bitfinex.com/ws/2",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close,
                              on_open = on_open)
    ws.run_forever()

--- request header ---
GET /ws/2 HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: api-pub.bitfinex.com
Origin: http://api-pub.bitfinex.com
Sec-WebSocket-Key: UYr0g1UQxmVrBX3gY3YR5A==
Sec-WebSocket-Version: 13
Cookie: __cfduid=dc0aafba6407ec0f2f8e5d04388967faa1549478650


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 07 Feb 2019 04:51:26 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: Wp9BkYp+BrMwhh281hcrZksHC7o=
uWebSockets: v0.15
X-Frame-Options: SAMEORIGIN
X-XSS-Protection: 1; mode=block
X-Content-Type-Options: nosniff
Strict-Transport-Security: max-age=31536000; includeSubdomains;
Expect-CT: max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"
Server: cloudflare
CF-RAY: 4a533dc61ad57e2d-DTW
-----------------------
send: b'\x81\xcd\x8f\xb1\xf3\x92\xf4\x91\xd1\xf7\xf9\xd4\x9d\xe6\xad\x8b\xd3\xb0\xfc\xc4\x91\xe1\xec\xc3\x9a\xf0\xea\x93\xdf\xb2\xad\xd2\x9b\xf3\xe1\xdf\x96\xfe\xad\x8b\x

'NoneType' object has no attribute 'connected'
### closed ###
